In [1]:
%reload_ext autoreload
%autoreload 2

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from rtseg.cells.plotting import generate_fork_plot, generate_abins_lbins, get_bulk_init_area
import numpy as np
import sys
from pathlib import Path
from skimage.io import imsave, imread
import zarr
from numcodecs import Zlib
from rtseg.utils.db_ops import read_from_db
from tifffile import imwrite # used for finer compression things
import h5py
import pandas as pd
from os.path import join
import polars as pl
%matplotlib qt5
from rtseg.utils.param_io import load_params
from rtseg.utils.disk_ops import read_files
import os
from skimage.metrics import structural_similarity as ssim
import pysal.lib
import esda.moran 
import pysal.explore
from scipy import stats
from scipy.spatial.distance import pdist, squareform 
from scipy.fft import fft2, fftfreq
from scipy.special import rel_entr

In [3]:
param = load_params(Path('C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0119/expt_params.yaml'), ref_type='expt')
param.Save.directory = 'C:/Users/oscbr226/Desktop/Initiation_Lib_Tweezer/Experiments/EXP-25-CD0119'
save_dir = Path(param.Save.directory)

In [4]:
use_pyarrow = True
columns_to_extract = ['area', 'length', 'normalized_internal_x', 'normalization_counts', 'position', 'trap']
all_data = pl.read_parquet(save_dir / Path('forks_all.parquet'), use_pyarrow=use_pyarrow, columns=columns_to_extract)

areas = all_data['area'].to_numpy()
longs = all_data['normalized_internal_x'].to_numpy()
lengths = all_data['length'].to_numpy()
counts = all_data['normalization_counts'].to_numpy()

bin_scale = param.Forkplots.bin_scale
heatmap_threshold = param.Forkplots.heatmap_threshold
pixel_size = param.Forkplots.pixel_size

heatmap, mean_cell_lengths, abins, lbins, (x, y) = generate_fork_plot(areas, lengths, longs, counts,
                bin_scale=bin_scale,
                pixel_size=pixel_size,
                heatmap_threshold=heatmap_threshold)



In [5]:
min_length = param.Forkplots.min_length
arb_div_area = param.Forkplots.arb_div_area
init_area = param.Forkplots.init_area_cv

init_area = get_bulk_init_area(areas, counts, longs, lengths, min_length=min_length, pixel_size=pixel_size, arb_div_area=arb_div_area)


In [6]:
plt.matshow(heatmap, aspect='auto', interpolation='none', extent=[x[0], x[-1], y[-1], y[0]], origin='upper', cmap='jet')
plt.plot(-0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.plot(0.5 * mean_cell_lengths, y, 'w', linewidth=2)
plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
plt.xlim(-3, 3) 
plt.ylim(3,y[0])
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm$^2$)')

Text(0, 0.5, 'Cell size (µm$^2$)')

In [13]:
init_area_cv = param.Forkplots.init_area_cv
init_area_low = init_area - init_area_cv*init_area
init_area_high = init_area + init_area_cv*init_area

abins_inds_around_init = np.where(np.logical_and(init_area_low <= abins, abins <= init_area_high))
abins_inds_around_init = abins_inds_around_init[0]
abins_inds_around_init = np.append(abins_inds_around_init, abins_inds_around_init[-1]+1)
area_bins_around_init = abins[abins_inds_around_init]
mean_cell_length_around_init = mean_cell_lengths[abins_inds_around_init[0:-1]]
max_cell_length_around_init = np.max(mean_cell_length_around_init)
lbins_inds_around_init = np.where(np.logical_and(-0.5*max_cell_length_around_init <= lbins , lbins <= 0.5*max_cell_length_around_init))
lbins_inds_around_init = lbins_inds_around_init[0]
lbins_inds_around_init = np.insert(lbins_inds_around_init, 0, [lbins_inds_around_init[0]-2, lbins_inds_around_init[0]-1])
lbins_inds_around_init = np.append(lbins_inds_around_init, [lbins_inds_around_init[-1]+1, lbins_inds_around_init[-1]+2])
lbins_around_init = lbins[lbins_inds_around_init]

heatmap_around_init = heatmap[np.ix_(abins_inds_around_init, lbins_inds_around_init)]

plot_abins_init = (area_bins_around_init[0:-1] + area_bins_around_init[1:])/2
plt.matshow(heatmap_around_init, aspect='auto', interpolation='none', 
            extent=[lbins_around_init[0], lbins_around_init[-1], area_bins_around_init[-1], area_bins_around_init[0]], origin='upper', cmap='jet')
plt.plot(-0.5 * mean_cell_length_around_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.plot(0.5 * mean_cell_length_around_init, y[abins_inds_around_init[0:-1]], 'w', linewidth=2)
plt.axhline(init_area, color='red', linestyle='--', linewidth=2)
[plt.axhline(y, color='magenta', linestyle='--') for y in y_centers]
plt.xlabel('Cell long axis (µm)')
plt.ylabel('Cell size (µm$^2$)')
plt.ylim(1.7, 1.35)



[-1.74678622 -1.64696986 -1.54715351 -1.44733715 -1.3475208  -1.24770444
 -1.14788809 -1.04807173 -0.94825538 -0.84843902 -0.74862267 -0.64880631
 -0.54898995 -0.4491736  -0.34935724 -0.24954089 -0.14972453 -0.04990818
  0.04990818  0.14972453  0.24954089  0.34935724  0.4491736   0.54898995
  0.64880631  0.74862267  0.84843902  0.94825538  1.04807173  1.14788809
  1.24770444  1.3475208   1.44733715  1.54715351  1.64696986  1.74678622] [1.67541973 1.6320175  1.58861527 1.54521304 1.50181081 1.45840858
 1.41500635 1.37160412]


(1.7, 1.35)

In [25]:

all_pos_numbers = all_data['position'].to_numpy()
pos_inds = np.unique(all_pos_numbers)
all_trap_numbers = all_data['trap'].to_numpy()
trap_nrs = np.unique(all_trap_numbers)
trap_nrs= trap_nrs[~np.isnan(trap_nrs)]

heatmap_around_init_dim = np.shape(heatmap_around_init)
heatmap_around_init_traps = np.zeros(shape=(heatmap_around_init_dim[0], heatmap_around_init_dim[-1], len(pos_inds)*len(trap_nrs)))
heatmap_around_init_pos_traps = np.zeros(shape=(heatmap_around_init_dim[0], heatmap_around_init_dim[-1], len(pos_inds), len(trap_nrs)))

k = 0
for i in range(0, len(pos_inds)):
    for j in range(0, len(trap_nrs)):
        print(f'Position {pos_inds[i]} Trap {trap_nrs[j]+1}')  
        trap_inds = np.where((all_pos_numbers == pos_inds[i]) & (all_trap_numbers == trap_nrs[j]))
        areas_trap = areas[trap_inds]
        longs_trap = longs[trap_inds]
        lengths_trap = lengths[trap_inds]
        counts_trap = counts[trap_inds]

        heatmap_trap, _, _, _, _ = generate_fork_plot(areas_trap, lengths_trap, longs_trap, counts_trap, abins=abins, lbins=lbins, 
                                                      bin_scale=bin_scale, pixel_size=pixel_size, heatmap_threshold=heatmap_threshold)

        heatmap_trap_init = heatmap_trap[np.ix_(abins_inds_around_init, lbins_inds_around_init)]
        heatmap_around_init_traps[:,:,k] = heatmap_trap_init
        heatmap_around_init_pos_traps[:,:,i,j] = heatmap_trap_init
        k += 1
        





Position 1 Trap 1.0
Position 1 Trap 2.0
Position 1 Trap 3.0
Position 1 Trap 4.0
Position 1 Trap 5.0
Position 1 Trap 6.0
Position 1 Trap 7.0
Position 1 Trap 8.0
Position 1 Trap 9.0
Position 1 Trap 10.0
Position 1 Trap 11.0
Position 1 Trap 12.0
Position 1 Trap 13.0
Position 1 Trap 14.0
Position 1 Trap 15.0
Position 1 Trap 16.0
Position 1 Trap 17.0
Position 1 Trap 18.0
Position 1 Trap 19.0
Position 1 Trap 20.0
Position 1 Trap 21.0
Position 1 Trap 22.0
Position 1 Trap 23.0
Position 1 Trap 24.0
Position 1 Trap 25.0
Position 1 Trap 26.0
Position 1 Trap 27.0
Position 1 Trap 28.0
Position 2 Trap 1.0
Position 2 Trap 2.0
Position 2 Trap 3.0
Position 2 Trap 4.0
Position 2 Trap 5.0
Position 2 Trap 6.0
Position 2 Trap 7.0
Position 2 Trap 8.0
Position 2 Trap 9.0
Position 2 Trap 10.0
Position 2 Trap 11.0
Position 2 Trap 12.0
Position 2 Trap 13.0
Position 2 Trap 14.0
Position 2 Trap 15.0
Position 2 Trap 16.0
Position 2 Trap 17.0
Position 2 Trap 18.0
Position 2 Trap 19.0
Position 2 Trap 20.0
Position 2

In [26]:
np.save(save_dir / Path('heatmaps_per_trap_around_init.npy'), heatmap_around_init_traps, heatmap_around_init_pos_traps)



In [ ]:
np.load(save_dir / Path('heatmaps_per_trap_around_init.npy'))

tot_nr_traps = heatmap_around_init_traps.shape[-1]

flat_heatmap_around_init = heatmap_around_init.flatten()

weight = pysal.lib.weights.lat2W(heatmap_around_init.shape[0], heatmap_around_init.shape[-1], rook=True) 

moran_flat_heatmap = esda.moran.Moran(flat_heatmap_around_init, weight)

#Calculations for the energy test
x_centers = np.linspace(lbins_around_init[0], lbins_around_init[-1], num=36)
y_centers = np.linspace(area_bins_around_init[-1], area_bins_around_init[0], num=8)
xx, yy = np.meshgrid(x_centers, y_centers)
bin_centers = np.column_stack((xx.ravel(), yy.ravel()))
dists = squareform(pdist(bin_centers))

#### All traps, no position information scoring

In [28]:
corrcoeffs = np.zeros(shape=(tot_nr_traps, ))
all_ssims = np.zeros(shape=(tot_nr_traps, ))
all_moran_bivars = np.zeros(shape=(tot_nr_traps, 3))
ks_results = np.zeros(shape=(tot_nr_traps, 2))
#chi_results = np.zeros(shape=(tot_nr_traps,2))
sobolev_results = np.zeros(shape=(tot_nr_traps, ))
energy_test_results = np.zeros(shape=(tot_nr_traps, ))

for i in range(0, tot_nr_traps):
    flat_trap_heatmap = heatmap_around_init_traps[:,:,i].flatten()
    
    #Pearson correlation coefficient 
    corrcoeffs[i] = np.corrcoef(flat_heatmap_around_init, flat_trap_heatmap)[0, 1]

    #SSIM
    max_val = np.max([np.max(flat_heatmap_around_init), np.max(flat_trap_heatmap)])
    min_val = np.min([np.min(flat_heatmap_around_init), np.min(flat_trap_heatmap)])
    all_ssims[i] = ssim(flat_heatmap_around_init, flat_trap_heatmap, data_range=max_val-min_val)

    #Moran's I
    #moran_flat_heatmap_trap = esda.moran.Moran(flat_trap_heatmap, weight)
    #print(f"Moran's I: {moran_flat_heatmap_trap.I}, p-value: {moran_flat_heatmap_trap.p_norm}")
    moran_bivar_heatmap = esda.moran.Moran_BV(flat_heatmap_around_init, flat_trap_heatmap, weight)
    all_moran_bivars[i,0] = moran_bivar_heatmap.I
    all_moran_bivars[i,1] = moran_bivar_heatmap.p_sim
    all_moran_bivars[i,2] = moran_bivar_heatmap.z_sim

    #Kolmogorov-Smirnov
    ks_stat, ks_pval = stats.ks_2samp(flat_heatmap_around_init, flat_trap_heatmap)
    ks_results[i,0] = ks_stat
    ks_results[i,1] = ks_pval

    #chi_stats, chi_pval = stats.chisquare(flat_trap_heatmap, flat_heatmap_around_init)
    #chi_results[:,0] = chi_stats
    #chi_results[:,1]= chi_pval
    
    #Sobolev
    diff_heatmap = heatmap_around_init - all_trap_heatmaps_around_init[:,:,i]
    fft_diff_heatmap = fft2(diff_heatmap)
    ny, nx = diff_heatmap.shape
    kx = fftfreq(nx)
    ky = fftfreq(ny)

    kx_2d, ky_2d = np.meshgrid(kx, ky)
    k_squared = kx_2d**2 + ky_2d**2
    k_squared[0,0] = 1

    weighted_sum_heatmaps = np.sum(np.abs(fft_diff_heatmap)**2 / (1+4*np.pi**2*k_squared))
    sobolev_norm = np.sqrt(weighted_sum_heatmaps)
    sobolev_results[i] = sobolev_norm

    #Energy test
    n1, n2 = np.sum(flat_heatmap_around_init), np.sum(flat_trap_heatmap)
    w1, w2 = flat_heatmap_around_init/n1, flat_trap_heatmap/n2

    phi_11 = np.sum(np.outer(w1, w1) * dists)
    phi_22 = np.sum(np.outer(w2, w2) * dists)
    phi_12 = np.sum(np.outer(w1, w2) * dists)
    energy = phi_11 + phi_22 - 2*phi_12

    energy_test_results[i] = energy
    
    #print(f"Cross-Moran's I between full fork plot and per trap fork plot (trap: {i+1}): {moran_bivar_heatmap.I}, p-value: {moran_bivar_heatmap.p_sim}, 'z-score: {moran_bivar_heatmap.z_sim}")
    

In [41]:
plt.close('all')

plot_range = np.arange(1, tot_nr_traps+1, 1)

plt.figure(1)
plt.plot(plot_range, corrcoeffs, 'o')
plt.xlabel('Channel')
plt.ylabel('Pearson correlation coefficient') 

plt.figure(2)
plt.plot(plot_range, all_ssims, 'o')
plt.xlabel('Channel')
plt.ylabel('SSIM') 

plt.figure(3)
plt.plot(plot_range, all_moran_bivars[:,0], 'o')
plt.xlabel('Channel')
plt.ylabel('Cross-Moran\'s I')

plt.figure(4)
plt.plot(plot_range, all_moran_bivars[:,1], 'o')
plt.xlabel('Channel')
plt.ylabel('Cross-Moran\'s I, p-value')

plt.figure(5)
plt.plot(plot_range, all_moran_bivars[:,2], 'o')
plt.xlabel('Channel')
plt.ylabel('Cross-Moran\'s I, z-score')

plt.figure(6)
plt.plot(plot_range, ks_results[:,0], 'o')
plt.xlabel('Channel')
plt.ylabel('KS statistic')

plt.figure(7)
plt.plot(plot_range, ks_results[:,1], 'o')
plt.xlabel('Channel')
plt.ylabel('KS statistic p-value')

plt.figure(8)
#plt.plot(plot_range, sobolev_results, 'o')
plt.plot(plot_range, sobolev_results/(1+sobolev_results), 'o')
plt.xlabel('Channel')
plt.ylabel('Sobolev norm')

plt.figure(9)
plt.plot(plot_range, energy_test_results, 'o')
plt.xlabel('Channel')
plt.ylabel('Energy test result')

Text(0, 0.5, 'Energy test result')

#### Position information scoring

In [39]:
nr_pos = len(pos_inds)
nr_traps = len(trap_nrs)
corrcoeffs_pos_trap = np.zeros(shape=(nr_pos, nr_traps))
all_ssims_pos_trap = np.zeros(shape=(nr_pos, nr_traps))
all_moran_bivars_pos_trap = np.zeros(shape=(nr_pos, nr_traps, 3))
ks_results_pos_trap = np.zeros(shape=(nr_pos, nr_traps, 2))
#chi_results = np.zeros(shape=(tot_nr_traps,2))
sobolev_results_pos_trap = np.zeros(shape=(nr_pos, nr_traps))
energy_test_results_pos_trap = np.zeros(shape=(nr_pos, nr_traps))

for i in range(0, len(pos_inds)):
    for j in range(0, len(trap_nrs)):
        flat_pos_trap_heatmap = heatmap_around_init_pos_traps[:,:,i,j].flatten()

        #Correlation coefficient 
        corrcoeffs_pos_trap[i, j] = np.corrcoef(flat_heatmap_around_init, flat_pos_trap_heatmap)[0, 1]

        #SSIM
        max_val = np.max([np.max(flat_heatmap_around_init), np.max(flat_pos_trap_heatmap)])
        min_val = np.min([np.min(flat_heatmap_around_init), np.min(flat_pos_trap_heatmap)])
        all_ssims_pos_trap[i,j] = ssim(flat_heatmap_around_init, flat_pos_trap_heatmap, data_range=max_val-min_val)
        
        #Moran's I
        moran_bivar_heatmap_pos_trap = esda.moran.Moran_BV(flat_heatmap_around_init, flat_pos_trap_heatmap, weight)
        all_moran_bivars_pos_trap[i,j,0] = moran_bivar_heatmap_pos_trap.I
        all_moran_bivars_pos_trap[i,j,1] = moran_bivar_heatmap_pos_trap.p_sim
        all_moran_bivars_pos_trap[i,j,2] = moran_bivar_heatmap_pos_trap.z_sim

        #Kolmogorov-Smirnov
        ks_stat_pos_trap, ks_pval_pos_trap = stats.ks_2samp(flat_heatmap_around_init, flat_pos_trap_heatmap)
        ks_results_pos_trap[i,j,0] = ks_stat_pos_trap
        ks_results_pos_trap[i,j,1] = ks_pval_pos_trap

        #Sobolev
        diff_heatmap = heatmap_around_init - heatmap_around_init_pos_traps[:,:,i,j]
        fft_diff_heatmap = fft2(diff_heatmap)
        ny, nx = diff_heatmap.shape
        kx = fftfreq(nx)
        ky = fftfreq(ny)

        kx_2d, ky_2d = np.meshgrid(kx, ky)
        k_squared = kx_2d**2 + ky_2d**2
        k_squared[0,0] = 1

        weighted_sum_heatmaps = np.sum(np.abs(fft_diff_heatmap)**2 / (1+4*np.pi**2*k_squared))
        sobolev_norm = np.sqrt(weighted_sum_heatmaps)
        sobolev_results_pos_trap[i,j] = sobolev_norm

        #Energy test
        n1, n2 = np.sum(flat_heatmap_around_init), np.sum(flat_pos_trap_heatmap)
        w1, w2 = flat_heatmap_around_init/n1, flat_pos_trap_heatmap/n2

        phi_11 = np.sum(np.outer(w1, w1) * dists)
        phi_22 = np.sum(np.outer(w2, w2) * dists)
        phi_12 = np.sum(np.outer(w1, w2) * dists)
        energy = phi_11 + phi_22 - 2*phi_12

        energy_test_results_pos_trap[i,j] = energy

In [54]:
plt.close('all')

plt.figure(11)
plt.imshow(corrcoeffs_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='Pearson correlation coefficient')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(12)
plt.imshow(all_ssims_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='SSIM')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(13)
plt.imshow(all_moran_bivars_pos_trap[:,:,0], cmap='viridis', aspect='auto')
plt.colorbar(label='Cross-Moran\'s I')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(14)
plt.imshow(all_moran_bivars_pos_trap[:,:,1], cmap='viridis', aspect='auto')
plt.colorbar(label='Cross-Moran\'s I, p-value')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(15)
plt.imshow(all_moran_bivars_pos_trap[:,:,2], cmap='viridis', aspect='auto')
plt.colorbar(label='Cross-Moran\'s I, z-score')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(16)
plt.imshow(ks_results_pos_trap[:,:,0], cmap='viridis_r', aspect='auto')
plt.colorbar(label='KS statistic')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(17)
plt.imshow(ks_results_pos_trap[:,:,1], cmap='viridis', aspect='auto')
plt.colorbar(label='KS statistic, p-value')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(18)
plt.imshow(sobolev_results_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='Sobolev norm')
plt.xlabel('Trap')
plt.ylabel('Position')

plt.figure(19)
plt.imshow(energy_test_results_pos_trap, cmap='viridis', aspect='auto')
plt.colorbar(label='Energy test result')
plt.xlabel('Trap')
plt.ylabel('Position')


Text(0, 0.5, 'Position')

#### Checking if metrics are similar 

In [53]:
plt.close('all')

plt.figure()
plt.plot(corrcoeffs, all_ssims, 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('SSIM')

plt.figure()
plt.plot(corrcoeffs, all_moran_bivars[:,0], 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('Cross-Moran\'s I')

plt.figure()
plt.plot(corrcoeffs, ks_results[:,0], 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('KS statistics')

plt.figure()
plt.plot(corrcoeffs, sobolev_results, 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('Sobolev norm')

plt.figure()
plt.plot(corrcoeffs, energy_test_results, 'o')
plt.xlabel('Pearson correlation coefficient')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(all_ssims, all_moran_bivars[:,0], 'o')
plt.xlabel('SSIM')
plt.ylabel('Cross-Moran\'s I')

plt.figure()
plt.plot(all_ssims, ks_results[:,0], 'o')
plt.xlabel('SSIM')
plt.ylabel('KS statistics')

plt.figure()
plt.plot(all_ssims, sobolev_results, 'o')
plt.xlabel('SSIM')
plt.ylabel('Sobolev norm')

plt.figure()
plt.plot(all_ssims, energy_test_results, 'o')
plt.xlabel('SSIM')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(all_moran_bivars[:,0], ks_results[:,0], 'o')
plt.xlabel('Cross-Moran\'s I')
plt.ylabel('KS statistics')

plt.figure()
plt.plot(all_moran_bivars[:,0], sobolev_results, 'o')
plt.xlabel('Cross-Moran\'s I')
plt.ylabel('Sobolev norm')
           
plt.figure()
plt.plot(all_moran_bivars[:,0], energy_test_results, 'o')
plt.xlabel('Cross-Moran\'s I')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(ks_results[:,0], sobolev_results, 'o')
plt.xlabel('KS statistics')
plt.ylabel('Sobolev norm')

plt.figure()
plt.plot(ks_results[:,0], energy_test_results, 'o')
plt.xlabel('KS statistics')
plt.ylabel('Energy test results')

plt.figure()
plt.plot(sobolev_results, energy_test_results, 'o')
plt.xlabel('Sobolev norm')
plt.ylabel('Energy test results')

Text(0, 0.5, 'Energy test results')